In [1]:
import os
import sys

import numpy as np
import pandas as pd
from sqlalchemy import create_engine


sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('../..'))))
from strategy import EquityStrategy
from tester import Tester


base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [2]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [3]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

In [4]:
total_ret_idx = pd.read_csv(eindex_path, header=0, index_col=0, parse_dates=True)
total_ret = total_ret_idx.pct_change(1)
prc_idx = pd.read_csv(eindex1_path, header=0, index_col=0, parse_dates=True)
prc_ret = prc_idx.pct_change(1)
fut_idx = pd.read_csv(efuture_path, header=0, index_col=0, parse_dates=True)
#fut_idx.index = pd.to_datetime(fut_idx.index, unit='D', origin=pd.Timestamp('1899-12-30'))
fut_ret = fut_idx.pct_change(1)

# 만약에 eindex에 빈 정보가 있으면 eindex1으로 대체
no_data_dates = total_ret[(total_ret.isnull().sum(axis=1) > 0).values].index
total_ret.loc[no_data_dates] = prc_ret.loc[no_data_dates]

# 2007년까지는 eindex 2008년부터는 future 사용
ERet = pd.concat([total_ret.loc[:'2007-12-31'], fut_ret.loc['2008-01-01':]], axis=0)
ERet.fillna(0, inplace=True)
Eindex = (1. + ERet).cumprod()

ERet.index.name = 'tdate'
Eindex.index.name = 'tdate'

ERet.columns.name = 'ticker'
Eindex.columns.name = 'ticker'

In [5]:
growthvalue = pd.read_csv(os.path.join(data_path, "growthvalue.csv"), index_col=0, parse_dates=True)
growthvalue.index.name = 'tdate'
growthvalue.columns.name = 'ticker'

In [6]:
minobs1=12
nopos=0.4
cs=0.35
per=3

In [7]:
i = 0

In [8]:
ret = ERet
index = Eindex

In [11]:
class EEM(EquityStrategy):
    def __init__(self, strategy_name, asset_type):
        super().__init__(strategy_name=strategy_name, asset_type=asset_type)
        self.err = None

    def load_strategy_data(self, table='datastream', origin='ERR'):
        self.err = self._load_strategy_data(table=table, origin=origin)

    def calculate_signal(self, minobs1=12, nopos=0.4, CS=0.35, longlen=12, shortlen=6, lag=0):
        """

        :param cs_num: percentage of position for cross sectional signal
        :param min_obs:
        :param longlen: long term price momentum period
        :param shortlen: short term price momentum period
        :return:
        """
        self.logger.info('[STEP 3] CACULATE SIGNAL')

        for i in range(2):
            if i == 0:
                RET = self.ret.loc[:, ['SPX', 'TSX', 'FTSE', 'DAX', 'CAC', 'SMI', 'MIB', 'IBEX', 'OMX', 'AEX']]
            else:
                RET = self.ret.loc[:, ['NKY', 'AS51', 'HSI', "SG"]]

            # 1. Load Data
            err = self.err.copy()

            Mag = err - err.rolling(window=longlen + 1).mean()
            Mag = Mag.iloc[longlen:]
            Mag = Mag[RET.columns]

            STDEV = Mag.rolling(window=shortlen).mean()
            STDEV = STDEV.iloc[shortlen - 1:]

            RV = STDEV
            RV1 = RV.iloc[:len(RV) - lag]
            RV1.index = RV.index[lag:]
            RV = RV1

            # 4. Rank
            pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
            RVrank = RV.expanding().apply(pctrank, raw=True)  # it takes some time
            RVrank = RVrank.iloc[minobs1 - 1:, ]

            # 5. Long Short
            truecount = (RVrank.notnull().sum(axis=1) * CS).apply(round)
            tiebreaker = RVrank.rolling(5).mean().fillna(0) * 0.0000001

            # 1. Cross sectional
            CSRV = (RVrank + tiebreaker).rank(axis=1, method='first')  # Short
            CSRV1 = (-1 * RVrank - 1 * tiebreaker).rank(axis=1, method='first')  # Long

            CSRVpos = CSRV.fillna(0) * 0
            CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
            CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1

            # Final CS signal
            CSRV = CSRVpos

            # 2. Time Series
            TSRV = RVrank.fillna(0) * 0
            TSRV[RVrank > nopos + (1 - nopos) / 2] = 1  # Long
            TSRV[RVrank < (1 - nopos) / 2] = -1  # Short

            if i == 0:
                TSRVrun1 = TSRV
                CSRVrun1 = CSRV
            else:
                TSRVrun2 = TSRV
                CSRVrun2 = CSRV

        TSRV = pd.concat([TSRVrun1, TSRVrun2], axis=1)
        CSRV = pd.concat([CSRVrun1, CSRVrun2], axis=1)

        TSRV = TSRV[self.ret.columns]
        CSRV = CSRV[self.ret.columns]

        self.TSRV = TSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')
        self.CSRV = CSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')

        # Align dates with each other
        if self.TSRV.index[0] > self.CSRV.index[0]:
            self.CSRV = self.CSRV.loc[self.TSRV.index[0]:]
        else:
            self.TSRV = self.TSRV.loc[self.CSRV.index[0]:]


In [13]:
i = 0
minobs1=12
nopos=0.4
CS=0.35
longlen=12
shortlen=6
lag=0

In [14]:
err = pd.read_csv(os.path.join(data_path, "ERR.csv"), index_col=0, parse_dates=True)
err.index.name = 'tdate'
err.columns.name = 'ticker'

In [16]:
if i == 0:
    RET = ret.loc[:, ['SPX', 'TSX', 'FTSE', 'DAX', 'CAC', 'SMI', 'MIB', 'IBEX', 'OMX', 'AEX']]
else:
    RET = ret.loc[:, ['NKY', 'AS51', 'HSI', "SG"]]

# 1. Load Data
err = err.copy()

Mag = err - err.rolling(window=longlen + 1).mean()
Mag = Mag.iloc[longlen:]
Mag = Mag[RET.columns]

In [18]:
STDEV = Mag.rolling(window=shortlen).mean()
STDEV = STDEV.iloc[shortlen - 1:]

RV = STDEV
RV1 = RV.iloc[:len(RV) - lag]
RV1.index = RV.index[lag:]
RV = RV1

In [19]:
RV.head()

ticker,SPX,TSX,FTSE,DAX,CAC,SMI,MIB,IBEX,OMX,AEX
tdate,,,,,,,,,,
1989-06-30,-35.192308,-27.230769,44.705128,241.794872,190.756410,245.397436,5.205128,357.538462,118.858974,NaN
1989-07-31,-47.871795,-12.987179,61.653846,95.205128,68.038462,265.384615,40.512821,152.846154,102.538462,NaN
1989-08-31,-77.679487,-56.000000,-8.692308,63.782051,35.538462,190.512821,-32.282051,328.717949,47.192308,NaN
1989-09-29,-104.500000,-50.679487,-8.474359,46.256410,7.538462,194.846154,50.551282,114.358974,5.397436,NaN
1989-10-31,-102.435897,-53.589744,-75.423077,7.794872,-84.846154,236.089744,-5.538462,-36.474359,-55.833333,NaN


In [22]:
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
RVrank = RV.expanding().apply(pctrank, raw=True)  # it takes some time
RVrank = RVrank.iloc[minobs1 - 1:, ]

# 3-2. Long Short
RV1 = RV.iloc[minobs1 - 1:]
truecount = (RV1.notnull().sum(axis=1) * CS).apply(round)

CSRV = (RV1).rank(axis=1, method='first')  # Short
CSRV1 = (-1 * RV1).rank(axis=1, method='first')  # Long

CSRVpos = CSRV.fillna(0) * 0
CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1
CSRV = CSRVpos

In [23]:
CSRV.head()

ticker,SPX,TSX,FTSE,DAX,CAC,SMI,MIB,IBEX,OMX,AEX
tdate,,,,,,,,,,
1990-05-31,0.0,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.0,0.0
1990-06-29,0.0,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.0,0.0
1990-07-31,0.0,-1.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1990-08-31,1.0,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0
1990-09-28,1.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,-1.0,0.0


In [ ]:


# 2. Time Series
TSRV = RVrank.fillna(0) * 0
TSRV[RVrank > nopos + (1 - nopos) / 2] = 1  # Long
TSRV[RVrank < (1 - nopos) / 2] = -1  # Short

if i == 0:
    TSRVrun1 = TSRV
    CSRVrun1 = CSRV
else:
    TSRVrun2 = TSRV
    CSRVrun2 = CSRV

TSRV = pd.concat([TSRVrun1, TSRVrun2], axis=1)
CSRV = pd.concat([CSRVrun1, CSRVrun2], axis=1)

TSRV = TSRV[ret.columns]
CSRV = CSRV[ret.columns]

TSRV = TSRV.loc[ret.index].fillna(method='ffill').dropna(how='all')
CSRV = CSRV.loc[ret.index].fillna(method='ffill').dropna(how='all')

# Align dates with each other
    if TSRV.index[0] > CSRV.index[0]:
CSRV = CSRV.loc[TSRV.index[0]:]
    else:
TSRV = TSRV.loc[CSRV.index[0]:]